In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# read content of book
book_raw = pd.read_csv("../data/processed/Prinz_raw.csv")

# copy raw content as a list to fist column second row   
book = book_raw.copy()

In [2]:
## layout of book = [{'raw':[], 'meta': {}, 'word_list': [], 'word_bag': {}, '2-gram': [], '3-gram': []}] in seperate csv saved at the end of the notebook
# Note - per Hand entfert: , " da Probleme mit pd.read_csv
# Note - per Hand bereinigt: Inhaltsverzeichnis, erste letzte Seite, Kopf und Fußzeilen

In [16]:
## tidy book

# convert everything to string
book = book.applymap(str)           # it should be string anyway

# save concat all rows in book to get a very long sentence without line breaks
sentence = ''.join(book['content'].tolist())

# remove: numbers
sentence = ''.join([i for i in sentence if not i.isdigit()])

# remove . : ! ? "  

# lower case everyting

print(sentence)

Wohin wollen sie? - Der Zugführer weiß es selbst nicht sagte der Weichensteller. Und ein erleuchteter Schnellzug donnerte in die entgegengesetzte Richtung vorbei. - Sie kommen schon zurück? fragte der kleine Prinz ... - Das sind nicht dieselben sagte der Weichensteller. Sie wurden ausgetauscht. - Waren sie nicht zufrieden dort wo sie waren? - Man ist niemals dort zufrieden wo man ist sagte der Weichen-steller. Und der Donner eines dritten Schnellzugs grollte. - Verfolgen sie die ersten Reisenden? fragte der kleine Prinz. - Sie verfolgen gar nichts sagte der Weichensteller. Sie schlafen im Inneren oder sie gähnen. Nur die Kinder drücken ihre Nasen an den Scheiben platt. - Nur die Kinder wissen wohin sie wollen sagte der kleine Prinz. Sie wenden ihre Zeit an eine Stoffpuppe und sie wächst ihnen so ans Herz und nimmt man sie ihnen weg dann weinen sie ... - Sie haben es gut sagte der Weichensteller. - Guten Tag sagte der kleine Prinz. - Guten Tag sagte der Händler. Er handelte mit durststi

In [4]:
## book_list

In [5]:
## book_bag

In [6]:
## book_meta

In [7]:
## book_2gram

In [8]:
## book_3gram

In [9]:
book_list.to_csv("../data/processed/Prinz_list.csv", index = False)
book_bag.to_csv("../data/processed/Prinz_bag.csv", index = False)
book_meta.to_csv("../data/processed/Prinz_meta.csv", index = False)
book_2gram.to_csv("../data/processed/Prinz_2gram.csv", index = False)
book_3gram.to_csv("../data/processed/Prinz_3gram.csv", index = False)

NameError: name 'book_list' is not defined